In [1]:
%pip install bitshuffle

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import blimpy as bl
#from ultralytics import YOLO
import time
import os
import random
import psutil
import scipy.ndimage
from PIL import Image
from PIL import ImageDraw
import imageio
%matplotlib inline

In [3]:
df = pd.read_csv('/home/cgchoza/galaxies/complete_cadences_catalog.csv')
df

,Target,Session,Band,Cadence ID,Frequency,.h5 path,.dat path,Time
0,DDO210,AGBT18A_999_103,L,24777,2251,/datag/pipeline/AGBT18A_999_103/collate/splice...,/home/obs/turboseti/AGBT18A_999_103/collate/sp...,2018-07-07 08:49:26
1,DDO210,AGBT18A_999_103,L,24777,2251,/datag/pipeline/AGBT18A_999_103/collate/splice...,/home/obs/turboseti/AGBT18A_999_103/collate/sp...,2018-07-07 08:49:26
2,DDO210,AGBT18A_999_103,L,24777,2251,/datag/pipeline/AGBT18A_999_103/collate/splice...,/home/obs/turboseti/AGBT18A_999_103/collate/sp...,2018-07-07 08:49:26
3,DDO210,AGBT18A_999_103,L,24777,2251,/datag/pipeline/AGBT18A_999_103/collate/splice...,/home/obs/turboseti/AGBT18A_999_103/collate/sp...,2018-07-07 08:49:26
4,DDO210,AGBT18A_999_103,L,24777,2251,/datag/pipeline/AGBT18A_999_103/collate/splice...,/home/obs/turboseti/AGBT18A_999_103/collate/sp...,2018-07-07 08:49:26
...,...,...,...,...,...,...,...,...
36553,NGC3226,AGBT22B_999_25,L,411390,1126,/datag/pipeline/AGBT22B_999_25/blc16_blp06/blc...,/home/obs/turboseti/AGBT22B_999_25/blc16_blp06...,2022-11-19 06:13:36
36554,NGC3226,AGBT22B_999_25,L,411390,1126,/datag/pipeline/AGBT22B_999_25/blc16_blp06/blc...,/home/obs/turboseti/AGBT22B_999_25/blc16_blp06...,2022-11-19 06:13:36
36555,NGC3226,AGBT22B_999_25,L,411390,1126,/datag/pipeline/AGBT22B_999_25/blc16_blp06/blc...,/home/obs/turboseti/AGBT22B_999_25/blc16_blp06...,2022-11-19 06:13:36
36556,NGC3226,AGBT22B_999_25,L,411390,1126,/datag/pipeline/AGBT22B_999_25/blc16_blp06/blc...,/home/obs/turboseti/AGBT22B_999_25/blc16_blp06...,2022-11-19 06:13:36


In [4]:
high_path = df['.h5 path'].iloc[-1]

In [5]:
df['.h5 path'] = df['.h5 path'].str.replace('0000.h5', '0002.h5', regex=False)
df = df.drop_duplicates(subset='.h5 path', keep='first').reset_index(drop=True)
df

,Target,Session,Band,Cadence ID,Frequency,.h5 path,.dat path,Time
0,DDO210,AGBT18A_999_103,L,24777,2251,/datag/pipeline/AGBT18A_999_103/collate/splice...,/home/obs/turboseti/AGBT18A_999_103/collate/sp...,2018-07-07 08:49:26
1,DDO210,AGBT18A_999_103,L,24777,2251,/datag/pipeline/AGBT18A_999_103/collate/splice...,/home/obs/turboseti/AGBT18A_999_103/collate/sp...,2018-07-07 08:49:26
2,DDO210,AGBT18A_999_103,L,24777,2251,/datag/pipeline/AGBT18A_999_103/collate/splice...,/home/obs/turboseti/AGBT18A_999_103/collate/sp...,2018-07-07 08:49:26
3,DDO210,AGBT18A_999_103,L,24777,2251,/datag/pipeline/AGBT18A_999_103/collate/splice...,/home/obs/turboseti/AGBT18A_999_103/collate/sp...,2018-07-07 08:49:26
4,DDO210,AGBT18A_999_103,L,24777,2251,/datag/pipeline/AGBT18A_999_103/collate/splice...,/home/obs/turboseti/AGBT18A_999_103/collate/sp...,2018-07-07 08:49:26
...,...,...,...,...,...,...,...,...
30309,NGC3226,AGBT22B_999_25,L,411390,1126,/datag/pipeline/AGBT22B_999_25/blc16_blp06/blc...,/home/obs/turboseti/AGBT22B_999_25/blc16_blp06...,2022-11-19 06:13:36
30310,NGC3226,AGBT22B_999_25,L,411390,1126,/datag/pipeline/AGBT22B_999_25/blc16_blp06/blc...,/home/obs/turboseti/AGBT22B_999_25/blc16_blp06...,2022-11-19 06:13:36
30311,NGC3226,AGBT22B_999_25,L,411390,1126,/datag/pipeline/AGBT22B_999_25/blc16_blp06/blc...,/home/obs/turboseti/AGBT22B_999_25/blc16_blp06...,2022-11-19 06:13:36
30312,NGC3226,AGBT22B_999_25,L,411390,1126,/datag/pipeline/AGBT22B_999_25/blc16_blp06/blc...,/home/obs/turboseti/AGBT22B_999_25/blc16_blp06...,2022-11-19 06:13:36


In [6]:
df = df[~df['.h5 path'].str.contains('spliced')].reset_index(drop=True)
df

,Target,Session,Band,Cadence ID,Frequency,.h5 path,.dat path,Time
0,AND_XIV,AGBT18B_999_07,S,30225,3151,/datag/pipeline/AGBT18B_999_07/blc40_blp00/blc...,/home/obs/turboseti/AGBT18B_999_07/blc40_blp00...,2018-08-18 08:41:37
1,AND_XIV,AGBT18B_999_07,S,30225,3151,/datag/pipeline/AGBT18B_999_07/blc40_blp00/blc...,/home/obs/turboseti/AGBT18B_999_07/blc40_blp00...,2018-08-18 08:41:37
2,AND_XIV,AGBT18B_999_07,S,30225,3151,/datag/pipeline/AGBT18B_999_07/blc40_blp00/blc...,/home/obs/turboseti/AGBT18B_999_07/blc40_blp00...,2018-08-18 08:41:37
3,AND_XIV,AGBT18B_999_07,S,30225,3151,/datag/pipeline/AGBT18B_999_07/blc40_blp00/blc...,/home/obs/turboseti/AGBT18B_999_07/blc40_blp00...,2018-08-18 08:41:37
4,AND_XIV,AGBT18B_999_07,S,30225,3151,/datag/pipeline/AGBT18B_999_07/blc40_blp00/blc...,/home/obs/turboseti/AGBT18B_999_07/blc40_blp00...,2018-08-18 08:41:37
...,...,...,...,...,...,...,...,...
29341,NGC3226,AGBT22B_999_25,L,411390,1126,/datag/pipeline/AGBT22B_999_25/blc16_blp06/blc...,/home/obs/turboseti/AGBT22B_999_25/blc16_blp06...,2022-11-19 06:13:36
29342,NGC3226,AGBT22B_999_25,L,411390,1126,/datag/pipeline/AGBT22B_999_25/blc16_blp06/blc...,/home/obs/turboseti/AGBT22B_999_25/blc16_blp06...,2022-11-19 06:13:36
29343,NGC3226,AGBT22B_999_25,L,411390,1126,/datag/pipeline/AGBT22B_999_25/blc16_blp06/blc...,/home/obs/turboseti/AGBT22B_999_25/blc16_blp06...,2022-11-19 06:13:36
29344,NGC3226,AGBT22B_999_25,L,411390,1126,/datag/pipeline/AGBT22B_999_25/blc16_blp06/blc...,/home/obs/turboseti/AGBT22B_999_25/blc16_blp06...,2022-11-19 06:13:36


In [7]:
#path = df['.h5 path'].iloc[-1]

In [8]:
# fb_high = bl.Waterfall(high_path)
# high_data = fb_high.data # shape: (279; 1; 65,536)
# plt.imshow(10*np.log10(high_data).squeeze(), aspect='auto')
# #plt.xlim(1250, 1251)
# plt.show()

In [9]:
# fb_mid = bl.Waterfall(path)
# mid_data = fb_mid.data # shape: (279; 1; 65,536)
# plt.imshow(10*np.log10(mid_data).squeeze(), aspect='auto')
# plt.xlim(10000, 10003)
# plt.show()

In [10]:
# fb_mid = bl.Waterfall(path)
# mid_data = fb_mid.data # shape: (279; 1; 65,536)
# plt.imshow(10*np.log10(mid_data).squeeze(), aspect='auto')
# plt.xlim(1250, 1251)
# plt.show()

In [11]:
df = df.drop(index = 17546)
small_df = df.sample(n=100, random_state=42).reset_index(drop=True)

In [12]:
# problem_path = small_df['.h5 path'].iloc[692]
# print(df[df['.h5 path'] == problem_path].index)

In [13]:
# fb = bl.Waterfall(problem_path, load_data=True)
# fb.plot_waterfall(cmap='viridis', figsize=(10, 5), title='Waterfall Plot')

In [ ]:
def normalize_data_zscore(data):
    """
    Convert to dB scale, z-score normalize, scale full dynamic range to 8-bit.
    Returns an 8-bit unsigned integer array.
    """
    data_db = 10 * np.log10(data)
    z = (data_db - np.mean(data_db)) / np.std(data_db)

    z_min = np.min(z)
    z_max = np.max(z)

    # Avoid divide-by-zero
    if z_max == z_min:
        return np.zeros_like(z, dtype=np.uint8)

    z_scaled = (z - z_min) / (z_max - z_min)
    return (255 * z_scaled).astype(np.uint8)

def compute_yolo_boxes(binary, orig_h, orig_w, plot_h, plot_w):
    """Label connected regions and convert to YOLO format"""
    labeled, num_features = scipy.ndimage.label(binary)
    slices = scipy.ndimage.find_objects(labeled)

    labels = []
    for sl in slices:
        t0, t1 = sl[0].start, sl[0].stop
        f0, f1 = sl[1].start, sl[1].stop

        # Calculate box in array (data) coordinates
        box_cx = (f0 + f1) / 2
        box_cy = (t0 + t1) / 2
        box_w = f1 - f0
        box_h = t1 - t0

        # Normalize to the data array size (NOT the plotted pixel size!)
        x_center = box_cx / orig_w
        y_center = box_cy / orig_h
        w = box_w / orig_w
        h = box_h / orig_h

        labels.append(f"0 {x_center:.6f} {y_center:.6f} {w:.6f} {h:.6f}")
    return labels


def draw_yolo_boxes(image_path, label_path, save_path):
    """Overlay YOLO boxes on image and save."""
    img = Image.open(image_path).convert("RGB")
    draw = ImageDraw.Draw(img)
    w, h = img.size

    with open(label_path, "r") as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) != 5:
                continue
            _, x_c, y_c, bw, bh = map(float, parts)
            x_c *= w
            y_c *= h
            bw *= w
            bh *= h
            x0 = int(x_c - bw / 2)
            y0 = int(y_c - bh / 2)
            x1 = int(x_c + bw / 2)
            y1 = int(y_c + bh / 2)
            draw.rectangle([x0, y0, x1, y1], outline="red", width=2)

    img.save(save_path)
    return save_path


In [15]:
# Assume both dfs have a common unique identifier column like 'filename' or 'img_path'
# Replace this column name if different
key_column = '.h5 path'

# Exclude any rows in df that are in small_df
df_remaining = df[~df[key_column].isin(small_df[key_column])]

# Randomly select 100 rows from what's left
df_subset = df_remaining.sample(n=100, random_state=42).reset_index(drop=True)


In [ ]:
# Shuffle and split
total_indices = list(range(len(small_df)))
random.seed(42)
random.shuffle(total_indices)

split_idx = int(0.8 * len(total_indices))
train_indices = set(total_indices[:split_idx])
val_indices = set(total_indices[split_idx:])

# Create all needed dirs
base_dir = '/datax/scratch/jliang/hand_label_masking_dataset_nozscore'
image_train_dir = os.path.join(base_dir, 'train/images')
label_train_dir = os.path.join(base_dir, 'train/mask_labels')
image_val_dir = os.path.join(base_dir, 'val/images')
label_val_dir = os.path.join(base_dir, 'val/mask_labels')
visualization_train_dir = os.path.join(base_dir, 'visualization/train')
visualization_val_dir = os.path.join(base_dir, 'visualization/val')
waterfall_train_dir = os.path.join(base_dir, 'waterfall/train')
waterfall_val_dir = os.path.join(base_dir, 'waterfall/val')

for d in [image_train_dir, label_train_dir, image_val_dir, label_val_dir]:
    os.makedirs(d, exist_ok=True)

# Factor for thresholding
factor = 7

empty_label_count = 0
empty_label_names = []

for index in range(0, len(small_df)):
    # Wait for memory if needed
    mem_threshold = 40 * (1024 ** 3)  # 40 GB
    while psutil.virtual_memory().available < mem_threshold:
        print('Waiting for memory to free up...')
        time.sleep(180)

    # Choose correct folder based on split
    if index in train_indices:
        image_dir = image_train_dir
        label_dir = label_train_dir
        w_dir = waterfall_train_dir
        vis_dir = visualization_train_dir
    else:
        image_dir = image_val_dir
        label_dir = label_val_dir
        w_dir = waterfall_val_dir
        vis_dir = visualization_val_dir
    h5_path = small_df['.h5 path'].iloc[index]

    # Load data
    fb = bl.Waterfall(h5_path, load_data=True)
        # Load and process
    data = fb.data.squeeze()
    data_db = 10 * np.log10(data) # used for visualization
    #data = normalize_data_zscore(data)

    # File paths
    name = f"img_{index:05d}.png"
    label_path = os.path.join(label_dir, name.replace(".png", ".txt"))
    w_path = os.path.join(w_dir, name)
    vis_path = os.path.join(vis_dir, name)
    os.makedirs(os.path.dirname(vis_path), exist_ok=True)

    # Ensure waterfall output directory exists
    os.makedirs(w_dir, exist_ok=True)

    # Plot with actual dB values
    plt.imshow(data_db, aspect='auto', cmap='viridis')
    plt.axis('off')
    plt.subplots_adjust(left=0, right=1, top=1, bottom=0)
    plt.savefig(w_path, bbox_inches='tight', pad_inches=0)
    plt.close()

    # Mask = keep everything
    mask = np.ones_like(data, dtype=bool)

    # Stats from masked data
    # Define coarse channel width (depends on your setup; usually 1024 or 128)
    coarse_channel_width = 1024  # change if needed
    num_coarse = data.shape[1] // coarse_channel_width

    # Mask out edge channels from each coarse block
    bad_channels = []
    for i in range(num_coarse):
        edge_bins = list(range(i * coarse_channel_width, i * coarse_channel_width + 5))  # 5-bin edge mask
        bad_channels.extend(edge_bins)

    # Apply masking
    data_db[:, bad_channels] = np.median(data_db)

    masked_values = data_db[mask]
    median = np.median(masked_values)
    std = np.std(masked_values)

    # Threshold to identify hits
    threshold = median + factor * std
    binary = data_db > threshold  # not mask
    # Merge nearby frequency hits (widen horizontally)
    # Fast dilation using a max filter (approximates dilation)
    binary = scipy.ndimage.maximum_filter(binary, size=(5, 600))

    # --- Load saved image to get correct dimensions ---
    img = Image.open(w_path)
    plot_w, plot_h = img.size
    orig_w, orig_h = data.shape[1], data.shape[0]  # Original dimensions

    # --- Recompute YOLO boxes using actual image size ---
    labels = compute_yolo_boxes(binary, orig_h, orig_w, plot_h, plot_w)
    # Save labels
    label_name = f'img_{index:05d}.txt'
    label_path = os.path.join(label_dir, label_name)
    with open(label_path, "w") as f:
        f.write("\n".join(labels))
    if len(labels) == 0:
        empty_label_count += 1
        empty_label_names.append(label_name)

    # # Save normalized image from dB-scaled data
    # img_uint8 = normalize_data_zscore(data_db)
    # Image.fromarray(data_db).convert("RGB").save(w_path)

    # Draw boxes
    draw_yolo_boxes(w_path, label_path, vis_path)

    print(f"Processed index: {index}; Data shape: {data.shape}")

# Optional: write summary
summary_path = os.path.join(base_dir, "empty_labels_summary.txt")
with open(summary_path, "w") as f:
    f.write(f"Total empty: {empty_label_count} / {len(df)}\n")
    for name in empty_label_names:
        f.write(f"{name}\n")

Processed index: 0
Processed index: 1
Processed index: 2
Processed index: 3
Processed index: 4
Processed index: 5
Processed index: 6
Processed index: 7
Processed index: 8
Processed index: 9
Processed index: 10
Processed index: 11
Processed index: 12
Processed index: 13
Processed index: 14
Processed index: 15
Processed index: 16
Processed index: 17
Processed index: 18
Processed index: 19
Processed index: 20
Processed index: 21
Processed index: 22
Processed index: 23
Processed index: 24
Processed index: 25
Processed index: 26
Processed index: 27
Processed index: 28
Processed index: 29
Processed index: 30
Processed index: 31
Processed index: 32
Processed index: 33
Processed index: 34
Processed index: 35
Processed index: 36
Processed index: 37
Processed index: 38
Processed index: 39
Processed index: 40
Processed index: 41
Processed index: 42
Processed index: 43
Processed index: 44
Processed index: 45
Processed index: 46
Processed index: 47
Processed index: 48
Processed index: 49
Processed 

In [17]:
# # Shuffle and split
# total_indices = list(range(len(small_df)))
# random.seed(42)
# random.shuffle(total_indices)

# split_idx = int(0.8 * len(total_indices))
# train_indices = set(total_indices[:split_idx])
# val_indices = set(total_indices[split_idx:])

# # Create all needed dirs
# base_dir = '/datax/scratch/jliang/hand_label_masking_dataset'
# image_train_dir = os.path.join(base_dir, 'train/images')
# label_train_dir = os.path.join(base_dir, 'train/mask_labels')
# image_val_dir = os.path.join(base_dir, 'val/images')
# label_val_dir = os.path.join(base_dir, 'val/mask_labels')
# visualization_train_dir = os.path.join(base_dir, 'visualization/train')
# visualization_val_dir = os.path.join(base_dir, 'visualization/val')
# waterfall_train_dir = os.path.join(base_dir, 'waterfall/train')
# waterfall_val_dir = os.path.join(base_dir, 'waterfall/val')

# for d in [image_train_dir, label_train_dir, image_val_dir, label_val_dir]:
#     os.makedirs(d, exist_ok=True)

# # Factor for thresholding
# factor = 7

# empty_label_count = 0
# empty_label_names = []

# for index in range(len(df) - 1, len(df)):
#     # Wait for memory if needed
#     mem_threshold = 40 * (1024 ** 3)  # 40 GB
#     while psutil.virtual_memory().available < mem_threshold:
#         print('Waiting for memory to free up...')
#         time.sleep(180)

#     # Choose correct folder based on split
#     if index in train_indices:
#         image_dir = image_train_dir
#         label_dir = label_train_dir
#         w_dir = waterfall_train_dir
#         vis_dir = visualization_train_dir
#     else:
#         image_dir = image_val_dir
#         label_dir = label_val_dir
#         w_dir = waterfall_val_dir
#         vis_dir = visualization_val_dir
#     h5_path = df['.h5 path'].iloc[index]
    
#     # Load data
#     fb = bl.Waterfall(h5_path, load_data=True)
#         # Load and process
#     data = fb.data.squeeze()
#     data_db = 10 * np.log10(data) # used for visualization
#     data = normalize_data_zscore(data)

#     # File paths
#     name = f"img_{index:05d}.png"
#     label_path = os.path.join(label_dir, name.replace(".png", ".txt"))
#     w_path = os.path.join(w_dir, name)
#     vis_path = os.path.join(vis_dir, name)
#     os.makedirs(os.path.dirname(vis_path), exist_ok=True)

#     # Ensure waterfall output directory exists
#     os.makedirs(w_dir, exist_ok=True)

#     # Plot with actual dB values
#     plt.imshow(data_db, aspect='auto', cmap='viridis')
#     plt.axis('off')
#     plt.subplots_adjust(left=0, right=1, top=1, bottom=0)
#     plt.savefig(w_path, bbox_inches='tight', pad_inches=0)
#     plt.close()

#     # Mask = keep everything
#     mask = np.ones_like(data, dtype=bool)

#     # Stats from masked data
#     # Define coarse channel width (depends on your setup; usually 1024 or 128)
#     coarse_channel_width = 1024  # change if needed
#     num_coarse = data.shape[1] // coarse_channel_width

#     # Mask out edge channels from each coarse block
#     bad_channels = []
#     for i in range(num_coarse):
#         edge_bins = list(range(i * coarse_channel_width, i * coarse_channel_width + 5))  # 5-bin edge mask
#         bad_channels.extend(edge_bins)

#     # Apply masking
#     data[:, bad_channels] = np.median(data)

#     masked_values = data[mask]
#     median = np.median(masked_values)
#     std = np.std(masked_values)

#     # Threshold to identify hits
#     threshold = median + factor * std
#     binary = data > threshold  # not mask
#     # Merge nearby frequency hits (widen horizontally)
#     # Fast dilation using a max filter (approximates dilation)
#     binary = scipy.ndimage.maximum_filter(binary, size=(5, 600))

#     # --- Load saved image to get correct dimensions ---
#     img = Image.open(w_path)
#     plot_w, plot_h = img.size
#     orig_w, orig_h = data.shape[1], data.shape[0]  # Original dimensions

#     # --- Recompute YOLO boxes using actual image size ---
#     labels = compute_yolo_boxes(binary, orig_h, orig_w, plot_h, plot_w)
#     # Save labels
#     label_name = f'img_{index:05d}.txt'
#     label_path = os.path.join(label_dir, label_name)
#     with open(label_path, "w") as f:
#         f.write("\n".join(labels))
#     if len(labels) == 0:
#         empty_label_count += 1
#         empty_label_names.append(label_name)

#     # # Save normalized image from dB-scaled data
#     # img_uint8 = normalize_data_zscore(data_db)
#     # Image.fromarray(data_db).convert("RGB").save(w_path)

#     # Draw boxes
#     draw_yolo_boxes(w_path, label_path, vis_path)

#     print(f"Processed index: {index}")

# # Optional: write summary
# summary_path = os.path.join(base_dir, "empty_labels_summary.txt")
# with open(summary_path, "w") as f:
#     f.write(f"Total empty: {empty_label_count} / {len(df)}\n")
#     for name in empty_label_names:
#         f.write(f"{name}\n")


